In [ ]:
import asyncio
from random import randint
from typing import Annotated

from agent_framework import Agent, tool
from agent_framework.observability import configure_otel_providers, get_tracer
from opentelemetry.trace import SpanKind
from opentelemetry.trace.span import format_trace_id
from pydantic import Field
from agent_framework.azure import AzureOpenAIResponsesClient
from azure.identity.aio import AzureCliCredential

import os
from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True)

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model = os.getenv("AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME")

print("Project Endpoint: ", project_endpoint)
print("Model: ", model)

In [ ]:
@tool(approval_mode="never_require")
async def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    await asyncio.sleep(randint(0, 10) / 10.0)  # Simulate a network call
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."

In [ ]:
configure_otel_providers()

In [ ]:
questions = ["What's the weather in Amsterdam?",
             "and in Paris, and which is better?", "Why is the sky blue?"]

with get_tracer().start_as_current_span("Scenario: Agent Chat", kind=SpanKind.CLIENT) as current_span:
    print(
        f"Trace ID: {format_trace_id(current_span.get_span_context().trace_id)}")

    credential = AzureCliCredential()
    client = AzureOpenAIResponsesClient(
        project_endpoint=project_endpoint,
        deployment_name=model,
        credential=credential,
    )

    agent = client.as_agent(
        name="CustomBot",
        instructions="You are a helpful assistant that remembers our conversation.",
        tools=[get_weather],
        id="custom-bot-001",
    )

    thread = agent.create_session()

    for question in questions:
        print(f"\nUser: {question}")
        print(f"{agent.name}: ", end="")
        async for update in agent.run(
            question,
            session=thread,
            stream=True,
        ):
            if update.text:
                print(update.text, end="")